# Random Forest

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import random
from pandas import ExcelWriter

In [2]:
#importing the uncleaned data
data_uncleaned=pd.read_csv('Computational Data Without Identifiers.csv')
#importing the cleaned data
data=pd.read_csv(r'C:\Users\Grant\Desktop\Cleaned PERL Research data.csv', encoding='latin1')
Rows = 1246
Inst_List_data=pd.read_csv('List of Institutions.csv')

In [3]:
#Creates a function with two user inputs, one for the first column to be used, one for the last
def Replace_Number(Column_Low_Limit,Column_High_Limit):
#The Difference between the the user inputs
    Limit_Diff = Column_High_Limit-Column_Low_Limit
    index_fix = Column_Low_Limit
    a = 0
#Creates a matrix of zeros that matches the size of the data to be changes to numbers
    global Numeric_data
    Numeric_data = np.zeros((Rows, Limit_Diff+1))
    while Column_Low_Limit <= Column_High_Limit:
        Rows_Complete = 1
#Selects each individual row in a column
        for i in data.values[1:1246,Column_Low_Limit]:
        #Replaces 0s with a +1 or -1, depending on answers in the data
            if i == "Checked" or i == "Yes" or i == "Agree" or i == "Strongly agree" or i == "Very strongly supported" or i == "Strongly supported" or  i == "Somewhat supported" or i == "Often" or i == "Sometimes" or i == "Used Often" or i == "Used Sometimes" or i == "Used rarely":    
                a = 1
                Numeric_data[Rows_Complete-1,Column_Low_Limit-index_fix] = a
            if i == "Unchecked" or i == "No" or i == "Disagree" or i == "Strongly disgaree" or i == "Never" or i == "Not supported" or i == "Not used at all":
                a = -1
                Numeric_data[Rows_Complete-1,Column_Low_Limit-index_fix] = a
            if Rows_Complete + 1 == Rows:
                Column_Low_Limit = Column_Low_Limit + 1
            Rows_Complete = Rows_Complete + 1

In [4]:
Replace_Number(1,148)

In [5]:
def Random_Forest(Predict, Det_Low, Det_High, Trees):
    #i will count iterations
    i = 0
    #z will make sure that the column where the prediction is stored does not get ran
    z = Det_Low
    #creates a variable to store the best tree
    global Best_Tree
    Best_Tree = 0
    #matrix to randomly decide an answers effect on the outcome
    Ans_pred = ["Positive", "Negative", "None"]
    best_tree_number_correct = 0
    #creates an array to be changed to the trees
    Forest = np.zeros((Trees, Det_High - Det_Low))
    #ensures the row that is trying to be predicted is skipped
    while z <= Rows+10:
        if z == Predict:
            z+=1
        else:
            z+=1
            #Randomly generates a forest
            while i <= Trees - 1:
                i+=1
                row_index = 0
                column_index = 0
                while column_index <= Det_High-Det_Low-1:
                    Ans_pred_det = random.choice(Ans_pred)
                    if "Positive" == Ans_pred_det:
                        Forest[i-1, column_index] = 1
                        column_index+=1
                    if "Negative" == Ans_pred_det:
                        Forest[i-1, column_index] = -1
                        column_index+=1
                    if "None" == Ans_pred_det:
                        Forest[i-1, column_index] = 0
                        column_index+=1
            #guesses what the outcome for each tree is
            while row_index <= Trees - 1:
                Numeric_data_row_key = 0
                guess_row_key = 0
                guess = []
                number_correct = 0
                number_incorrect = 0
                while Numeric_data_row_key <= Rows - 1:           
                    guess_det_mat = Numeric_data[Numeric_data_row_key, Det_Low-1:Det_High-1]*Forest[row_index,:]
                    guess_det_int = np.sum(guess_det_mat)
                    Numeric_data_row_key+=1
                    if guess_det_int == 0:
                        guess.append("Other")
                    else:
                        if guess_det_int <= 0:
                            guess.append("Unchecked")
                        if guess_det_int >= 0:
                            guess.append("Checked")
                #determiness if the guess is correct in order to find the best tree
                for actual in data.values[:,8]:
                    if actual == guess[guess_row_key] or guess[guess_row_key] == "Other":
                        if actual == guess[guess_row_key]:
                            number_correct+=1
                            guess_row_key+=1
                        else:
                            if actual == "Checked" or actual == "Unchecked":
                                number_incorrect+=1
                                guess_row_key+=1
                            else:
                                number_correct+=1
                                guess_row_key+=1
                    else:
                        number_incorrect+=1
                #stores the best tree
                if number_correct >= best_tree_number_correct:
                    Best_Tree = Forest[row_index,:]
                    best_tree_number_correct = number_correct
                    print(best_tree_number_correct)
                    #if the tree passes a certain threshold, saves the matrix
                    if number_correct >= 1100:
                        df = pd.DataFrame(Best_Tree)
                        writer = ExcelWriter('Best Tree.xlsx')
                        df.to_excel(writer, 'Sheet1')
                        writer.save
                row_index+=1

In [6]:
#probably needs to be run about 10^7 times to get a good tree
Random_Forest(8,1,148,100)

329
347
358
367
384
385
